In [3]:
import numpy as np
import pandas as pd
from scipy import stats

In [5]:
import os, glob, pathlib
print("CWD:", os.getcwd())                  # 현재 작업 폴더
print("여기 있는 csv:", glob.glob("*.csv")) # 현재 폴더의 csv 목록

CWD: C:\Users\User
여기 있는 csv: []


In [7]:
df = pd.read_csv("boston_1978.csv")
df_num = df.select_dtypes(include=[np.number]).copy()

tax_col = [c for c in df_num.columns if c.strip().lower() == "tax"][0]
y = df_num[tax_col].to_numpy()

def pearson_manual(x, y):
    x_mean, y_mean = np.mean(x), np.mean(y)
    num = np.sum((x - x_mean)*(y - y_mean))
    den = np.sqrt(np.sum((x - x_mean)**2) * np.sum((y - y_mean)**2))
    return num/den

corr_rows = []
for col in df_num.columns:
    if col == tax_col:
        continue   # 자기 자신 제외
    r = pearson_manual(df_num[col].to_numpy(), y)
    corr_rows.append((col, r, abs(r)))

corr_table = (pd.DataFrame(corr_rows, columns=["variable","r_with_TAX","abs_r"])
                .sort_values("abs_r", ascending=False)
                .reset_index(drop=True))

top4_vars = corr_table.loc[:3, "variable"].tolist()
print("=== 절대값 |r|기준 상위 4개 변수 ===")
print(top4_vars)

=== 절대값 |r|기준 상위 4개 변수 ===
['RAD', 'TRACT', 'INDUS', 'NOX']


In [9]:
X = df_num[top4_vars].to_numpy()
X = np.column_stack((np.ones(len(X)), X))    # [1, X1, X2, X3, X4]

XtX = X.T @ X
XtX_inv = np.linalg.inv(XtX)
Xty = X.T @ y
beta_hat = XtX_inv @ Xty

# 예측/잔차 및 설명계수
y_hat = X @ beta_hat
resid = y - y_hat
n, p = X.shape                         # p에는 절편 포함
SSE = np.sum(resid**2)                 # 오차제곱합
SST = np.sum((y - np.mean(y))**2)      # 총제곱합
R2 = 1 - SSE/SST                       # 결정계수
Adj_R2 = 1 - ((1 - R2) * (n - 1) / (n - p))   #결정계수 조정

sigma2 = SSE / (n - p)                
var_beta = sigma2 * XtX_inv
se_beta = np.sqrt(np.diag(var_beta))
t_vals = beta_hat / se_beta
p_vals = 2*(1 - stats.t.cdf(np.abs(t_vals), df=n - p))   # 양측검정

In [12]:
print("\n=== (a) 4개 변수 유의성 판단 (α = 0.05) ===")
headline = f"{'변수':<10s}{'계수(β̂)':>15s}{'t-값':>15s}{'p-값':>15s}{'판정':>10s}"
print(headline)
names = ["const"] + top4_vars
for name, b, t, p in zip(names, beta_hat, t_vals, p_vals):
    if name == "const":
        continue   # 상수
    sig = "유의" if p < 0.05 else "비유의"
    print(f"{name:<10s}{b:>15.6f}{t:>15.3f}{p:>15.3g}{sig:>10s}")

print("\n=== (b) 4개의 변수를 이용한 회구모형  ===")
terms = [f"{beta_hat[i+1]:.4f}·{top4_vars[i]}" for i in range(4)]
eq = " + ".join(terms)
print(f"TAX = {beta_hat[0]:.4f} + {eq}")

print("\n=== (c) 회귀모형의 설명계수  ===")
print(f"R² = {R2:.6f}")
print(f"Adjusted R² = {Adj_R2:.6f}")   # 설명변수 2개이상임로 fair하게 쓸 설명계수



=== (a) 4개 변수 유의성 판단 (α = 0.05) ===
변수                 계수(β̂)            t-값            p-값        판정
RAD             13.638513         24.094              0        유의
TRACT           -0.006589         -1.908         0.0569       비유의
INDUS            6.642506         10.794              0        유의
NOX             -0.042076         -0.001          0.999       비유의

=== (b) 4개의 변수를 이용한 회구모형  ===
TAX = 221.8363 + 13.6385·RAD + -0.0066·TRACT + 6.6425·INDUS + -0.0421·NOX

=== (c) 회귀모형의 설명계수  ===
R² = 0.879040
Adjusted R² = 0.878074
